In [121]:
import pandas as pd
from autograd import grad 
import autograd.numpy as np
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt

### Data Preparation

In [98]:
data = np.genfromtxt('data.csv', delimiter=',')
x = data[:2]
y = data[-1][np.newaxis, :]
print(x.shape) #x is 2 features, 96 data points
print(y.shape)

(2, 96)
(1, 96)


In [99]:
scaler = StandardScaler()
#x is shape of (2,96)
#it needs to be transposed before standard scale to have batch normalization
#x_scaled is shape of (96,2) so it needs to be transposed back
x_scaled_intermediate = scaler.fit_transform(x.T) 
x_scaled = x_scaled_intermediate.T

#### StandardScaler Raw Code Study

In [93]:
#batch normalization across each data sample.
#[:,np.newaxis] is just to add a dimension
#x_means has shape (2,1) <- two means aligned in one column. 
#array([[0.47115037],
#       [0.54882423]])
#x_stds has shape (2,1)
x_means = np.mean(x,axis = 1)[:,np.newaxis]
x_stds = np.std(x,axis = 1)[:,np.newaxis]  

#for experiment, replace std with a small value
x_stds = np.array([[0.0026920197],[0.25804162]])


# if x_stds < small threshold, add 1 to the where std is small
# this is to prevent from dividing a very small std and the output numbers are very large
ind = np.argwhere(x_stds < 10**(-2))
if len(ind) > 0:
    ind = [v[0] for v in ind]
    adjust = np.zeros((x_stds.shape))
    adjust[ind] = 1.0
    x_stds += adjust

print(f"The adjusted x_stds is, ", x_stds)

# create standard normalizer function
normalizer = lambda data: (data - x_means)/x_stds

# create inverse standard normalizer
inverse_normalizer = lambda data: data*x_stds + x_means

The adjusted x_stds is,  [[1.00269202]
 [0.25804162]]


### Initialize W matrix

The final W matrix is a list with 2 elements [ , ] \
The second element is the weight for final linear combination in prediction function. \
The first element is another list, which contains the weight for each hidden layer. \
Each weight matrix is an numpy array. 


Here is the size assumptions: \
N = 2 (two input features) \
M = 1 (one prediction outcome) \
H_1 = 10; H_2 = 10; H_3 = 10 (number of units per hidden layer) 


Here is the w matrix size (#features, #units in next layer) \
$W_{input->h1}$ = (3, 10) Notice that #features = 3 because it's 2 input features + 1 bias \
$W_{h1->h2}$ = (11, 10) \
$W_{h2->h3}$ = (11, 10) \
$W_{h3->output}$ = (11, 1) 

W = [[$W_{input->h1}$, $W_{h1->h2}$, $W_{h2->h3}$],$W_{h3->output}$]

In [1]:
def initialize_network_weights(layer_sizes, scale):
    weights = []
    
    for k in range(len(layer_sizes)-1):
        # get layer sizes for current weight matrix
        U_k = layer_sizes[k]
        U_k_plus_1 = layer_sizes[k+1]

        # make weight matrix
        # np.random.randn follows normal distribution of mean 0 std 1
        # times scale here is to change the standard deviation to scale
        weight = scale*np.random.randn(U_k+1,U_k_plus_1)
        weights.append(weight)

    w_init = [weights[:-1],weights[-1]]
    
    return w_init

In [2]:
# An example 4 hidden layer network, with 10 units in each layer
N = 2  # dimension of input
M = 1  # dimension of output
U_1 = 10; U_2 = 10; U_3 = 10;  # number of units per hidden layer
# the list defines our network architecture
layer_sizes = [N, U_1,U_2,U_3,M]
# generate initial weights for our network
w_init = initialize_network_weights(layer_sizes, scale = 0.5)

NameError: name 'np' is not defined

In [178]:
print(f"W matrix size, ", len(w_init))
print(f"W first element size, ", len(w_init[0]))
print(f"W first elements shape, ", w_init[0][0].shape, w_init[0][1].shape, w_init[0][2].shape)
print(f"W second element shape, ", w_init[1].shape)

W matrix size,  2
W first element size,  3
W first elements shape,  (3, 10) (11, 10) (11, 10)
W second element shape,  (11, 1)


### Write Cost Function

In [181]:

###### two-class classification costs #######
# the convex softmax cost function
def softmax(w,x,y,iter):
    # compute cost over batch
    cost = np.sum(np.log(1 + np.exp(-y*model(x,w))))
    return cost/float(np.size(y))

def model(x,w):   
    # feature transformation - switch for dealing
    # with feature transforms that either do or do
    # not have internal parameters
    f = 0
    f = feature_transforms(x,w[0]) 

    # compute linear combination and return
    # switch for dealing with feature transforms that either 
    # do or do not have internal parameters
    a = 0
    a = w[1][0] + np.dot(f.T,w[1][1:])
    return a.T

def feature_transforms(a, w):    
    activation = lambda data: np.tanh(data)
    # loop through each layer matrix
    for W in w:
        # compute inner product with current layer weights
        a = W[0] + np.dot(a.T, W[1:])

        # output of layer activation
        a = activation(a).T
    return a

### Compute Gradient Descent

In [174]:
max_its = 1000
alpha_choice = 0.1

train_num = batch_size = np.size(y)
weight_histories = []
train_cost_histories = []

x_val = []
y_val = []
weight_history,train_cost_history,val_cost_history = super_optimizers_gradient_descent(mylib5.cost,w_init,x_scaled,y,x_val,y_val,alpha_choice,max_its,batch_size,verbose=False)
weight_histories.append(weight_history)
train_cost_histories.append(train_cost_history)


<function flatten_func.<locals>.<lambda> at 0x15408e670>


/var/folders/c3/jcvr35lj59lcykv3srmmv9p00000gn/T/ipykernel_77139/1468230978.py:92: RuntimeWarning: invalid value encountered in scalar divide
  return cost/float(np.size(y_p))


In [134]:
mylib5 = Setup(x,y)

mylib5.preprocessing_steps(normalizer = 'standard')

# split into training and validation sets
mylib5.make_train_val_split(train_portion = 1)

# choose cost
mylib5.choose_cost(name = 'softmax')

# choose dimensions of fully connected multilayer perceptron layers
layer_sizes = [10,10,10]
mylib5.choose_features(feature_name = 'multilayer_perceptron',layer_sizes = layer_sizes,activation = 'tanh',scale = 0.5)

# fit an optimization
mylib5.fit(max_its = 1000,alpha_choice = 10**(-1),verbose = False)

/var/folders/c3/jcvr35lj59lcykv3srmmv9p00000gn/T/ipykernel_77139/1468230978.py:92: RuntimeWarning: invalid value encountered in scalar divide
  return cost/float(np.size(y_p))
/var/folders/c3/jcvr35lj59lcykv3srmmv9p00000gn/T/ipykernel_77139/4149735772.py:160: RuntimeWarning: invalid value encountered in scalar divide
  val_accuracy_history = [1 - self.counter(v,self.x_val,self.y_val)/float(self.y_val.size) for v in weight_history]


### Reference Classes

In [123]:
from matplotlib.axes._axes import _log as matplotlib_axes_logger
matplotlib_axes_logger.setLevel('ERROR')
import autograd.numpy as np
from inspect import signature
from autograd import value_and_grad 
from autograd import hessian
from autograd.misc.flatten import flatten_func
from IPython.display import clear_output
from timeit import default_timer as timer
import time

#### Super Setup

In [120]:
class Setup:
    def __init__(self,x,y,**kwargs):
        # link in data
        self.x = x
        self.y = y
        
        # make containers for all histories
        self.weight_histories = []
        self.train_cost_histories = []
        self.train_accuracy_histories = []
        self.val_cost_histories = []
        self.val_accuracy_histories = []
        self.train_costs = []
        self.train_counts = []
        self.val_costs = []
        self.val_counts = []
        
    #### define preprocessing steps ####
    def preprocessing_steps(self,**kwargs):        
        ### produce / use data normalizer ###
        normalizer_name = 'standard'
        if 'normalizer_name' in kwargs:
            normalizer_name = kwargs['normalizer_name']
        self.normalizer_name = normalizer_name

        # produce normalizer / inverse normalizer
        s = normalizers_Setup(self.x,normalizer_name)
        self.normalizer = s.normalizer
        self.inverse_normalizer = s.inverse_normalizer
        
        # normalize input 
        self.x = self.normalizer(self.x)
       
    #### split data into training and validation sets ####
    def make_train_val_split(self,train_portion):
        # translate desired training portion into exact indecies
        self.train_portion = train_portion
        r = np.random.permutation(self.x.shape[1])
        train_num = int(np.round(train_portion*len(r)))
        self.train_inds = r[:train_num]
        self.val_inds = r[train_num:]
        
        # define training and testing sets
        self.x_train = self.x[:,self.train_inds]
        self.x_val = self.x[:,self.val_inds]

        self.y_train = self.y[:,self.train_inds]
        self.y_val = self.y[:,self.val_inds]
     
    #### define cost function ####
    def choose_cost(self,name,**kwargs):
        # create training and testing cost functions
        self.cost_object = super_cost_functions_Setup(name,**kwargs)

        # if the cost function is a two-class classifier, build a counter too
        if name == 'softmax' or name == 'perceptron':
            self.count_object = super_cost_functions_Setup('twoclass_counter',**kwargs)
                        
        if name == 'multiclass_softmax' or name == 'multiclass_perceptron':
            self.count_object = super_cost_functions_Setup('multiclass_counter',**kwargs)
  
        self.cost_name = name
    
    #### define feature transformation ####
    def choose_features(self,**kwargs): 
        ### select from pre-made feature transforms ###
        layer_sizes = [1]
        if 'layer_sizes' in kwargs:
            layer_sizes = kwargs['layer_sizes']
        
        # add input and output layer sizes
        input_size = self.x.shape[0]
        layer_sizes.insert(0, input_size)
      
        # add output size
        if self.cost_name == 'least_squares' or self.cost_name == 'least_absolute_deviations':
            layer_sizes.append(self.y.shape[0])
        else:
            num_labels = len(np.unique(self.y))
            if num_labels == 2:
                layer_sizes.append(1)
            else:
                layer_sizes.append(num_labels)
        
        # multilayer perceptron #
        feature_name = 'multilayer_perceptron'
        if 'name' in kwargs:
            feature_name = kwargs['feature_name']
           
        if feature_name == 'multilayer_perceptron':
            transformer = multilayer_perceptron_Setup(**kwargs)
            self.feature_transforms = transformer.feature_transforms
            self.multilayer_initializer = transformer.initializer
            self.layer_sizes = transformer.layer_sizes
            
        self.feature_name = feature_name
        
        ### with feature transformation constructed, pass on to cost function ###
        self.cost_object.define_feature_transform(self.feature_transforms)
        self.cost = self.cost_object.cost
        self.model = self.cost_object.model
        
        # if classification performed, inject feature transforms into counter as well
        if self.cost_name == 'softmax' or self.cost_name == 'perceptron' or self.cost_name == 'multiclass_softmax' or self.cost_name == 'multiclass_perceptron':
            self.count_object.define_feature_transform(self.feature_transforms)
            self.counter = self.count_object.cost
            
    #### run optimization ####
    def fit(self,**kwargs):
        # basic parameters for gradient descent run (default algorithm)
        max_its = 500; alpha_choice = 10**(-1);
        
        # set parameters by hand
        if 'max_its' in kwargs:
            self.max_its = kwargs['max_its']
        if 'alpha_choice' in kwargs:
            self.alpha_choice = kwargs['alpha_choice']
        
        # set initialization
        self.w_init = self.multilayer_initializer()
        
        # batch size for gradient descent?
        self.train_num = np.size(self.y_train)
        self.val_num = np.size(self.y_val)
        self.batch_size = np.size(self.y_train)
        if 'batch_size' in kwargs:
            self.batch_size = min(kwargs['batch_size'],self.batch_size)
        
        # verbose or not
        verbose = True
        if 'verbose' in kwargs:
            verbose = kwargs['verbose']

        # optimize
        weight_history = []
        cost_history = []
        
        # run gradient descent
        weight_history,train_cost_history,val_cost_history = super_optimizers_gradient_descent(self.cost,self.w_init,self.x_train,self.y_train,self.x_val,self.y_val,self.alpha_choice,self.max_its,self.batch_size,verbose=verbose)
                                                                                         
        # store all new histories
        self.weight_histories.append(weight_history)
        self.train_cost_histories.append(train_cost_history)
        self.val_cost_histories.append(val_cost_history)

        # if classification produce count history
        if self.cost_name == 'softmax' or self.cost_name == 'perceptron' or self.cost_name == 'multiclass_softmax' or self.cost_name == 'multiclass_perceptron':
            train_accuracy_history = [1 - self.counter(v,self.x_train,self.y_train)/float(self.y_train.size) for v in weight_history]
            val_accuracy_history = [1 - self.counter(v,self.x_val,self.y_val)/float(self.y_val.size) for v in weight_history]

            # store count history
            self.train_accuracy_histories.append(train_accuracy_history)
            self.val_accuracy_histories.append(val_accuracy_history)
    

#### super_optimizers_gradient_descent

In [ ]:
def super_optimizers_gradient_descent(g,w,x_train,y_train,x_val,y_val,alpha,max_its,batch_size,**kwargs): 
    verbose = True
    if 'verbose' in kwargs:
        verbose = kwargs['verbose']
    
    # flatten the input function, create gradient based on flat function
    g_flat, unflatten, w = flatten_func(g, w)
    grad = value_and_grad(g_flat)

    # record history
    num_train = y_train.size
    num_val = y_val.size
    w_hist = [unflatten(w)]
    train_hist = [g_flat(w,x_train,y_train,np.arange(num_train))]
    val_hist = [g_flat(w,x_val,y_val,np.arange(num_val))]

    # how many mini-batches equal the entire dataset?
    num_batches = int(np.ceil(np.divide(num_train, batch_size)))

    # over the line
    for k in range(max_its):                   
        # loop over each minibatch
        start = timer()
        train_cost = 0
        for b in range(num_batches):
            # collect indices of current mini-batch
            batch_inds = np.arange(b*batch_size, min((b+1)*batch_size, num_train))
            
            # plug in value into func and derivative
            cost_eval,grad_eval = grad(w,x_train,y_train,batch_inds)
            grad_eval.shape = np.shape(w)
    
            # take descent step with momentum
            w = w - alpha*grad_eval

        end = timer()
        
        # update training and validation cost
        train_cost = g_flat(w,x_train,y_train,np.arange(num_train))
        val_cost = g_flat(w,x_val,y_val,np.arange(num_val))

        # record weight update, train and val costs
        w_hist.append(unflatten(w))
        train_hist.append(train_cost)
        val_hist.append(val_cost)

        if verbose == True:
            print ('step ' + str(k+1) + ' done in ' + str(np.round(end - start,1)) + ' secs, train cost = ' + str(np.round(train_hist[-1][0],4)) + ', val cost = ' + str(np.round(val_hist[-1][0],4)))

    if verbose == True:
        print ('finished all ' + str(max_its) + ' steps')
        #time.sleep(1.5)
        #clear_output()
    return w_hist,train_hist,val_hist

#### super_cost_functions_Setup

In [124]:
class super_cost_functions_Setup:
    def __init__(self,name,**kwargs):        
        ### make cost function choice ###
        # for regression
        if name == 'least_squares':
            self.cost = self.least_squares
        if name == 'least_absolute_deviations':
            self.cost = self.least_absolute_deviations
            
        # for two-class classification
        if name == 'softmax':
            self.cost = self.softmax
        if name == 'perceptron':
            self.cost = self.perceptron
        if name == 'twoclass_counter':
            self.cost = self.counting_cost
            
        # for multiclass classification
        if name == 'multiclass_perceptron':
            self.cost = self.multiclass_perceptron
        if name == 'multiclass_softmax':
            self.cost = self.multiclass_softmax
        if name == 'multiclass_counter':
            self.cost = self.multiclass_counting_cost
            
        # for autoencoder
        if name == 'autoencoder':
            self.feature_transforms = feature_transforms
            self.feature_transforms_2 = kwargs['feature_transforms_2']
            self.cost = self.autoencoder
            
    ### insert feature transformations to use ###
    def define_feature_transform(self,feature_transforms):
        # make copy of feature transformation
        self.feature_transforms = feature_transforms
        
        # count parameter layers of input to feature transform
        self.sig = signature(self.feature_transforms)
            
    ##### models functions #####
    # compute linear combination of features
    def model(self,x,w):   
        # feature transformation - switch for dealing
        # with feature transforms that either do or do
        # not have internal parameters
        f = 0
        if len(self.sig.parameters) == 2:
            f = self.feature_transforms(x,w[0])
        else: 
            f = self.feature_transforms(x)    

        # compute linear combination and return
        # switch for dealing with feature transforms that either 
        # do or do not have internal parameters
        a = 0
        if len(self.sig.parameters) == 2:
            a = w[1][0] + np.dot(f.T,w[1][1:])
        else:
            a = w[0] + np.dot(f.T,w[1:])
        return a.T

    ###### regression costs #######
    # an implementation of the least squares cost function for linear regression
    def least_squares(self,w,x,y,iter):
        # get batch of points
        x_p = x[:,iter]
        y_p = y[:,iter]
                
        # compute cost over batch
        cost = np.sum((self.model(x_p,w) - y_p)**2)
        return cost/float(np.size(y_p))

    # a compact least absolute deviations cost function
    def least_absolute_deviations(self,w,x,y,iter):
        # get batch of points
        x_p = x[:,iter]
        y_p = y[:,iter]

        # compute cost over batch
        cost = np.sum(np.abs(self.model(x_p,w) - y_p))
        return cost/float(np.size(y_p))

    ###### two-class classification costs #######
    # the convex softmax cost function
    def softmax(self,w,x,y,iter):
        # get batch of points
        x_p = x[:,iter]
        y_p = y[:,iter]
        
        # compute cost over batch
        cost = np.sum(np.log(1 + np.exp(-y_p*self.model(x_p,w))))
        return cost/float(np.size(y_p))

    # the convex relu cost function
    def relu(self,w,x,y,iter):
        # get batch of points
        x_p = x[:,iter]
        y_p = y[:,iter]
        
        # compute cost over batch
        cost = np.sum(np.maximum(0,-y_p*self.model(x_p,w)))
        return cost/float(np.size(y_p))

    # the counting cost function
    def counting_cost(self,w,x,y):
        cost = np.sum((np.sign(self.model(x,w)) - y)**2)
        return 0.25*cost 

    ###### multiclass classification costs #######
    # multiclass perceptron
    def multiclass_perceptron(self,w,x,y,iter):
        # get subset of points
        x_p = x[:,iter]
        y_p = y[:,iter]

        # pre-compute predictions on all points
        all_evals = self.model(x_p,w)

        # compute maximum across data points
        a =  np.max(all_evals,axis = 0)        

        # compute cost in compact form using numpy broadcasting
        b = all_evals[y_p.astype(int).flatten(),np.arange(np.size(y_p))]
        cost = np.sum(a - b)

        # return average
        return cost/float(np.size(y_p))

    # multiclass softmax
    def multiclass_softmax(self,w,x,y,iter):     
        # get subset of points
        x_p = x[:,iter]
        y_p = y[:,iter]
        
        # pre-compute predictions on all points
        all_evals = self.model(x_p,w)

        # compute softmax across data points
        a = np.log(np.sum(np.exp(all_evals),axis = 0)) 

        # compute cost in compact form using numpy broadcasting
        b = all_evals[y_p.astype(int).flatten(),np.arange(np.size(y_p))]
        cost = np.sum(a - b)

        # return average
        return cost/float(np.size(y_p))

    # multiclass misclassification cost function - aka the fusion rule
    def multiclass_counting_cost(self,w,x,y):                
        # pre-compute predictions on all points
        all_evals = self.model(x,w)

        # compute predictions of each input point
        y_predict = (np.argmax(all_evals,axis = 0))[np.newaxis,:]

        # compare predicted label to actual label
        count = np.sum(np.abs(np.sign(y - y_predict)))

        # return number of misclassifications
        return count

#### normalizers_Setup

In [125]:
class normalizers_Setup:
    def __init__(self,x,name):
        normalizer = 0
        inverse_normalizer = 0
        
        # use standard normalizer
        if name == 'standard':
            self.normalizer, self.inverse_normalizer = self.standard_normalizer(x)
            
        # use PCA sphereing
        elif name == 'PCA_sphere':
            # create normalizer
            self.normalizer, self.inverse_normalizer = self.PCA_sphereing(x)
        
        # use ZCA sphereing
        elif name == 'ZCA_sphere':
            self.normalizer, self.inverse_normalizer = self.ZCA_sphereing(x)
            
        else:
            self.normalizer = lambda data: data
            self.inverse_normalizer = lambda data: data
            
    # standard normalization function 
    def standard_normalizer(self,x):
        # compute the mean and standard deviation of the input
        x_means = np.mean(x,axis = 1)[:,np.newaxis]
        x_stds = np.std(x,axis = 1)[:,np.newaxis]   
        
        # check to make sure thta x_stds > small threshold, for those not
        # divide by 1 instead of original standard deviation
        ind = np.argwhere(x_stds < 10**(-2))
        if len(ind) > 0:
            ind = [v[0] for v in ind]
            adjust = np.zeros((x_stds.shape))
            adjust[ind] = 1.0
            x_stds += adjust

        # create standard normalizer function
        normalizer = lambda data: (data - x_means)/x_stds

        # create inverse standard normalizer
        inverse_normalizer = lambda data: data*x_stds + x_means

        # return normalizer 
        return normalizer,inverse_normalizer

    # compute eigendecomposition of data covariance matrix for PCA transformation
    def PCA(self,x,**kwargs):
        # regularization parameter for numerical stability
        lam = 10**(-7)
        if 'lam' in kwargs:
            lam = kwargs['lam']

        # create the correlation matrix
        P = float(x.shape[1])
        Cov = 1/P*np.dot(x,x.T) + lam*np.eye(x.shape[0])

        # use numpy function to compute eigenvalues / vectors of correlation matrix
        d,V = np.linalg.eigh(Cov)
        return d,V

    # PCA-sphereing - use PCA to normalize input features
    def PCA_sphereing(self,x,**kwargs):
        # Step 1: mean-center the data
        x_means = np.mean(x,axis = 1)[:,np.newaxis]
        x_centered = x - x_means

        # Step 2: compute pca transform on mean-centered data
        d,V = self.PCA(x_centered,**kwargs)

        # Step 3: divide off standard deviation of each (transformed) input, 
        # which are equal to the returned eigenvalues in 'd'.  
        stds = (d[:,np.newaxis])**(0.5)
        
        # check to make sure thta x_stds > small threshold, for those not
        # divide by 1 instead of original standard deviation
        ind = np.argwhere(stds < 10**(-2))
        if len(ind) > 0:
            ind = [v[0] for v in ind]
            adjust = np.zeros((stds.shape))
            adjust[ind] = 1.0
            stds += adjust
        
        normalizer = lambda data: np.dot(V.T,data - x_means)/stds

        # create inverse normalizer
        inverse_normalizer = lambda data: np.dot(V,data*stds) + x_means

        # return normalizer 
        return normalizer,inverse_normalizer
    
    
    # ZCA-sphereing - use ZCA to normalize input features
    def ZCA_sphereing(self,x,**kwargs):
        # Step 1: mean-center the data
        x_means = np.mean(x,axis = 1)[:,np.newaxis]
        x_centered = x - x_means

        # Step 2: compute pca transform on mean-centered data
        d,V = self.PCA(x_centered,**kwargs)

        # Step 3: divide off standard deviation of each (transformed) input, 
        # which are equal to the returned eigenvalues in 'd'.  
        stds = (d[:,np.newaxis])**(0.5)
        
        # check to make sure thta x_stds > small threshold, for those not
        # divide by 1 instead of original standard deviation
        ind = np.argwhere(stds < 10**(-2))
        if len(ind) > 0:
            ind = [v[0] for v in ind]
            adjust = np.zeros((stds.shape))
            adjust[ind] = 1.0
            stds += adjust
             
        normalizer = lambda data: np.dot(V, np.dot(V.T,data - x_means)/stds)

        # create inverse normalizer
        inverse_normalizer = lambda data: np.dot(V,np.dot(V.T,data)*stds) + x_means

        # return normalizer 
        return normalizer,inverse_normalizer


#### multilayer_perceptron_Setup

In [126]:
class multilayer_perceptron_Setup:
    def __init__(self,**kwargs):  
        # set default values for layer sizes, activation, and scale
        activation = 'relu'

        # decide on these parameters via user input
        if 'activation' in kwargs:
            activation = kwargs['activation']

        # switches
        if activation == 'linear':
            self.activation = lambda data: data
        elif activation == 'tanh':
            self.activation = lambda data: np.tanh(data)
        elif activation == 'relu':
            self.activation = lambda data: np.maximum(0,data)
        elif activation == 'sinc':
            self.activation = lambda data: np.sinc(data)
        elif activation == 'sin':
            self.activation = lambda data: np.sin(data)
        elif activation == 'maxout':
            self.activation = lambda data1,data2: np.maximum(data1,data2)
                        
        # get layer sizes
        layer_sizes = kwargs['layer_sizes']
        self.layer_sizes = layer_sizes
        self.scale = 0.1
        if 'scale' in kwargs:
            self.scale = kwargs['scale']
            
        # assign initializer / feature transforms function
        if activation == 'linear' or activation == 'tanh' or activation == 'relu' or activation == 'sinc' or activation == 'sin':
            self.initializer = self.standard_initializer
            self.feature_transforms = self.standard_feature_transforms
        elif activation == 'maxout':
            self.initializer = self.maxout_initializer
            self.feature_transforms = self.maxout_feature_transforms

    ####### initializers ######
    # create initial weights for arbitrary feedforward network
    def standard_initializer(self):
        # container for entire weight tensor
        weights = []

        # loop over desired layer sizes and create appropriately sized initial 
        # weight matrix for each layer
        for k in range(len(self.layer_sizes)-1):
            # get layer sizes for current weight matrix
            U_k = self.layer_sizes[k]
            U_k_plus_1 = self.layer_sizes[k+1]

            # make weight matrix
            weight = self.scale*np.random.randn(U_k+1,U_k_plus_1)
            weights.append(weight)

        # re-express weights so that w_init[0] = omega_inner contains all 
        # internal weight matrices, and w_init = w contains weights of 
        # final linear combination in predict function
        w_init = [weights[:-1],weights[-1]]

        return w_init
    
    # create initial weights for arbitrary feedforward network
    def maxout_initializer(self):
        # container for entire weight tensor
        weights = []

        # loop over desired layer sizes and create appropriately sized initial 
        # weight matrix for each layer
        for k in range(len(self.layer_sizes)-1):
            # get layer sizes for current weight matrix
            U_k = self.layer_sizes[k]
            U_k_plus_1 = self.layer_sizes[k+1]

            # make weight matrix
            weight1 = self.scale*np.random.randn(U_k + 1,U_k_plus_1)

            # add second matrix for inner weights
            if k < len(self.layer_sizes)-2:
                weight2 = self.scale*np.random.randn(U_k + 1,U_k_plus_1)
                weights.append([weight1,weight2])
            else:
                weights.append(weight1)

        # re-express weights so that w_init[0] = omega_inner contains all 
        # internal weight matrices, and w_init = w contains weights of 
        # final linear combination in predict function
        w_init = [weights[:-1],weights[-1]]

        return w_init

    ####### feature transforms ######
    # fully evaluate our network features using the tensor of weights in w
    def standard_feature_transforms(self,a, w):    
        # loop through each layer matrix
        for W in w:
            # compute inner product with current layer weights
            a = W[0] + np.dot(a.T, W[1:])

            # output of layer activation
            a = self.activation(a).T
        return a
    
    # fully evaluate our network features using the tensor of weights in w
    def maxout_feature_transforms(self,a, w):    
        # loop through each layer matrix
        for W1,W2 in w:
            # compute inner product with current layer weights
            a1 = W1[0][:,np.newaxis] + np.dot(a.T, W1[1:]).T
            a2 = W2[0][:,np.newaxis] + np.dot(a.T, W2[1:]).T

            # output of layer activation
            a = self.activation(a1,a2)
        return a